## 1. Importing Libraries

In [2]:
import shutil
import apiquery
import pandas as pd
import sys
import seaborn as sns
import os
import numpy as np
import random
import torch
import gc
DATA_PATH = '../01.Data'
shutil.copy("apiquery_pyc.py", "apiquery.pyc")

module_path = "../src"
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils.training import *
from utils.encoding import *
from utils.utils import *
from utils.fetch import *
from dataset.dataset import BNPParibasText
from models.models import Roberta_Model
from utils.EarlyStopping import EarlyStopping
from utils.LoopFunctions import train_fn,valid_fn
from utils.prediction import get_prediction,get_embedding
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
import math
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import math
import time
import lightgbm as lgbm
import matplotlib.pyplot as plt
import torch.nn as nn
import config
import transformers

In [3]:
%%time
df_train     = pd.read_csv(os.path.join("../01.Data",'fold.csv'))
y_submission = pd.read_csv(os.path.join(DATA_PATH,'y_test_submission_example.tsv'), index_col='Index', encoding='utf-8', sep='\t')

CPU times: user 1.72 s, sys: 159 ms, total: 1.88 s
Wall time: 1.88 s


In [4]:
def calc_oof(df,config):
    df.loc[:,'oof'] = -1
    for fold in np.sort(df.fold.unique()):
        print(f'Predicting Model: {fold}')
        valid       = df[df['fold']==fold]
        valid_index = valid.index.to_list()
        valid = valid.reset_index(drop=True)
        # Defining DataSet
        col_unique = generate_col_unique(valid,config.COLUMNS_ENCODE)
        tokenizer = transformers.RobertaTokenizer.from_pretrained(config.PRETRAINED)
        valid_dataset = BNPParibasText(valid,config.MAX_LENGTH,tokenizer,col_unique)
        valid_loader = torch.utils.data.DataLoader(
            valid_dataset,
            batch_size  = config.BATCH_SIZE,
            num_workers = config.NUM_WORKERS,
            shuffle     = False,
            pin_memory  = True,
        )

       # Defining Device
        model = Roberta_Model(pretrained_model=config.PRETRAINED,dropout = config.DROPOUT)
        model.load_state_dict(torch.load(f'../03.Models/BNP_PARIBAS_ROBERTA_FOLD_{fold}'))
        model.to(config.DEVICE)
        preds = get_prediction(valid_loader, model,config.DEVICE)
        df.loc[valid_index,'oof'] = preds
    oof_score = np.sqrt(mean_squared_error(df['target'],df['oof']))
    print('OOF_SCORE (RMSE): ',oof_score)
    return oof_score


# Calculating predictions for test
def calculate_test(test,config):
    col_unique = generate_col_unique(test,config.COLUMNS_ENCODE)   
    tokenizer     = transformers.RobertaTokenizer.from_pretrained(config.PRETRAINED)
    test_dataset = BNPParibasText(test,config.MAX_LENGTH,tokenizer,col_unique)
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size  = config.BATCH_SIZE,
        pin_memory  = True,
        num_workers = config.NUM_WORKERS
    )
    preds = 0
    for fold in range(0,5):
        model = Roberta_Model(pretrained_model=config.PRETRAINED,dropout = config.DROPOUT)
        model.load_state_dict(torch.load(f'../03.Models/Roberta-Pretrained/BNP_PARIBAS_ROBERTA_FOLD_{fold}'))
        model.to(config.DEVICE)
        preds = preds + get_prediction(test_loader, model,config.DEVICE)
    test['preds'] = preds/5
    print(f'Real RMSE: ',math.sqrt(mean_squared_error(test['preds'].values,test['Target'].values)))

In [5]:
def run(data,fold,output_path,config,run=None):
    print(f'******************** Model Fold {fold}  *****************')
    seed_everything(seed=config.SEED)
    train = data[data['fold']!=fold].reset_index(drop=True)
    valid = data[data['fold']==fold].reset_index(drop=True)
    col_unique = generate_col_unique(train,config.COLUMNS_ENCODE)
    col_unique = generate_col_unique(valid,config.COLUMNS_ENCODE)
    
    
    print('Train: ',train.shape[0], 'Valid: ',valid.shape[0])
    # Defining DataSet
    tokenizer     = transformers.RobertaTokenizer.from_pretrained(config.PRETRAINED)
    train_dataset = BNPParibasText(train,config.MAX_LENGTH,tokenizer,col_unique)
    valid_dataset = BNPParibasText(valid,config.MAX_LENGTH,tokenizer,col_unique)
        
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size  = config.BATCH_SIZE,
        pin_memory  = True,
        num_workers = config.NUM_WORKERS
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size  = config.BATCH_SIZE,
        num_workers = config.NUM_WORKERS,
        shuffle     = False,
        pin_memory  = True,
    )
    
    # Defining Device
    model = Roberta_Model(pretrained_model=config.PRETRAINED,dropout = config.DROPOUT)
    model.to(config.DEVICE)
    criterion = nn.MSELoss()
    criterion.to(config.DEVICE)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": config.WEIGHT_DECAY,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int((len(train) / config.BATCH_SIZE )* config.EPOCHS)
    config.SCHEDULER_PARAMETERS['NUM_TRAIN_STEPS'] = num_train_steps
    print(f'num_train_steps: {num_train_steps}')
    optimizer = fetch_optimizer(config.OPTIMIZER_NAME,config.LEARNING_RATE,optimizer_parameters)
    scheduler = fetch_scheduler(config.SCHEDULER_NAME,optimizer,config.SCHEDULER_PARAMETERS)
    
    es = EarlyStopping (patience = config.EARLY_STOPPING, mode = config.MODE,delta=0)
      
    for epoch in range(config.EPOCHS):
        print('Epoch {}, lr {}'.format(epoch, optimizer.param_groups[0]['lr']))        
        training_loss = train_fn(train_loader,model,criterion,optimizer,config.DEVICE,scheduler,mode_sched = config.MODE_SCHEDULER)
        valid_loss    = valid_fn(valid_loader,model,criterion,config.DEVICE)
        if run:
            run.log({'training_loss':training_loss,'valid_loss':valid_loss})
            
        es(valid_loss, model,output_path)
        
        if es.early_stop:
            print('Meet early stopping')
            return es.get_best_val_score()
        gc.collect()
        torch.cuda.empty_cache()
    print("Didn't meet early stopping")
    return es.get_best_val_score()

In [7]:
for i in range(0,5):
    output_path = f'../03.Models/Roberta-Pretrained/BNP_PARIBAS_ROBERTA_FOLD_{i}'
    run(df_train,i,output_path,config)

******************** Model Fold 4  *****************
Train:  81623 Valid:  20405


  0%|          | 0/1276 [00:00<?, ?it/s]

num_train_steps: 15304
Epoch 0, lr 0.0
Mode Scheduler: OK


  0%|          | 0/319 [00:00<?, ?it/s]

Training -> Loss: 67.19512297292488


...

Epoch 11, lr 1.841149992739945e-06
Mode Scheduler: OK


  0%|          | 0/319 [00:00<?, ?it/s]

Training -> Loss: 14.920602123752284


100%|██████████| 319/319 [00:29<00:00, 10.75it/s, Eval_Loss=22.1]


Validation -> Loss: 22.061710692498377
Validation score improved (22.253296053895383 --> 22.061710692498377). Saving model!
Didn't meet early stopping


In [17]:
calc_oof(df_train,config)

Predicting Model: 0



Predicting Model: 1



Predicting Model: 2



Predicting Model: 3



Predicting Model: 4



OOF_SCORE (RMSE):  4.740007182216003


4.740007182216003

In [22]:
test           = pd.read_csv(os.path.join(DATA_PATH,'test_preprocessed.csv'))
test['target'] = -1
calculate_test(test,config)


Real RMSE:  4.6244272231443775


## Combining Word Embeddings with More Features

In [7]:
col_unique = generate_col_unique(df_train,config.COLUMNS_ENCODE)
tokenizer     = transformers.RobertaTokenizer.from_pretrained(config.PRETRAINED)
train_dataset = BNPParibasText(df_train,config.MAX_LENGTH,tokenizer,col_unique)
train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size  = 32,
            pin_memory  = True,
            num_workers = 72
        )
embedding_all = 0
for fold in np.sort(df_train['fold'].unique()):
    model         = Roberta_Model(pretrained_model=config.PRETRAINED)
    model.load_state_dict(torch.load(f'../03.Models/BNP_PARIBAS_ROBERTA_FOLD_{fold}'))
    embedding_all += get_embedding(train_loader, model, 'cuda')
    del model
    torch.cuda.empty_cache()
embedding_all = embedding_all/len(df_train['fold'].unique())
df_train[[f'emb_{col_unique}_{i}' for i in range(embedding_all.shape[1])]] = embedding_all

FileNotFoundError: [Errno 2] No such file or directory: '../03.Models/Roberta-Pretrained/BNP_PARIBAS_ROBERTA_FOLD_0'

In [47]:
df_test      = pd.read_csv(os.path.join(DATA_PATH,'test_preprocessed.csv'))
df_test['target'] = -1

col_unique = generate_col_unique(df_test,config.COLUMNS_ENCODE)
tokenizer     = transformers.RobertaTokenizer.from_pretrained(config.PRETRAINED)
test_dataset = BNPParibasText(df_test,config.MAX_LENGTH,tokenizer,col_unique)
test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size  = 32,
            pin_memory  = True,
            num_workers = 72
        )
embedding_all = 0
for fold in np.sort(df_train['fold'].unique()):
    model         = Roberta_Model(pretrained_model=config.PRETRAINED)
    model.load_state_dict(torch.load(f'../03.Models/Roberta-Pretrained/BNP_PARIBAS_ROBERTA_FOLD_{fold}'))
    embedding_all += get_embedding(test_loader, model, 'cuda')
    del model
    torch.cuda.empty_cache()
embedding_all = embedding_all/len(df_train['fold'].unique())
df_test[[f'emb_{col_unique}_{i}' for i in range(embedding_all.shape[1])]] = embedding_all

In [43]:
columns_modeling = ['additives_n','ingredients_from_palm_oil_n',
                    'ingredients_that_may_be_from_palm_oil_n','target',
                    'states_en_brands','states_en_categories','states_en_characteristics','states_en_expiration date',
                    'states_en_general_complete','states_en_ingredients','pnns_groups_1','pnns_groups_2',
                    'states_en_packaging','states_en_packaging-code-','states_en_photo_upload',
                    'states_en_photo_validate','states_en_product name','states_en_quantity','diff_t'] + [f'emb_{col_unique}_{i}' for i in range(embedding_all.shape[1])]
columns_label = df_train[columns_modeling].select_dtypes(include=['object']).columns.to_list()
print(columns_label)

['states_en_brands', 'states_en_categories', 'states_en_characteristics', 'states_en_expiration date', 'states_en_general_complete', 'states_en_ingredients', 'pnns_groups_1', 'pnns_groups_2', 'states_en_packaging', 'states_en_packaging-code-', 'states_en_photo_upload', 'states_en_photo_validate', 'states_en_product name', 'states_en_quantity']


In [44]:
df_train,dict_le = label_encoding(df_train,label_cols = columns_label, drop_original = True, missing_new_cat = True)

Mode: Missing as new category
Label Encoding:  label_states_en_brands
Label Encoding:  label_states_en_categories
Label Encoding:  label_states_en_characteristics
Label Encoding:  label_states_en_expiration date
Label Encoding:  label_states_en_general_complete
Label Encoding:  label_states_en_ingredients
Label Encoding:  label_pnns_groups_1
Label Encoding:  label_pnns_groups_2
Label Encoding:  label_states_en_packaging
Label Encoding:  label_states_en_packaging-code-
Label Encoding:  label_states_en_photo_upload
Label Encoding:  label_states_en_photo_validate
Label Encoding:  label_states_en_product name
Label Encoding:  label_states_en_quantity


In [48]:
df_test = apply_label_encoder(df_test,dict_le,drop_original = True, missing_new_cat = True)

Mode: Missing as new category
Applying Label Encoding:  label_states_en_brands
Applying Label Encoding:  label_states_en_categories
Applying Label Encoding:  label_states_en_characteristics
Applying Label Encoding:  label_states_en_expiration date
Applying Label Encoding:  label_states_en_general_complete
Applying Label Encoding:  label_states_en_ingredients
Applying Label Encoding:  label_pnns_groups_1
Applying Label Encoding:  label_pnns_groups_2
Applying Label Encoding:  label_states_en_packaging
Applying Label Encoding:  label_states_en_packaging-code-
Applying Label Encoding:  label_states_en_photo_upload
Applying Label Encoding:  label_states_en_photo_validate
Applying Label Encoding:  label_states_en_product name
Applying Label Encoding:  label_states_en_quantity


In [52]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'rmse'},
        'num_leaves':12,
        'learning_rate': 0.001,
        "min_child_samples": 150,
        "max_depth" : 5,
        'feature_fraction':  0.5,
        "bagging_freq": 1,
        'bagging_fraction': 0.75,
        "is_unbalance" : False,
        'force_col_wise':True,
        'num_threads':18,
        #"scale_pos_weight":5 -> Generally  is the ratio of number of negative class to the positive class.
        'bagging_seed':42,
        'lambda_l1':1.5,
        'lambda_l2':1,
        'verbose': 1

}
cat_columns = [i for i in df_train.columns.to_list() if i.startswith('label_')]
columns_modeling_last = list(set(columns_modeling)-set(columns_label)) + ['fold'] + cat_columns 

In [53]:
results,models,importances,oof,feature_list = Training_Lightgbm(df_train[columns_modeling_last],params,fold_column = 'fold',target_column = 'target',cat_vars = cat_columns ,metric = 'RMSE',early_stopping = 200,max_boost_round = 8000)

Columns: ['emb_product_name_ingredients_text_brands_tags__557', 'emb_product_name_ingredients_text_brands_tags__730', 'emb_product_name_ingredients_text_brands_tags__746', 'emb_product_name_ingredients_text_brands_tags__2', 'emb_product_name_ingredients_text_brands_tags__638', 'emb_product_name_ingredients_text_brands_tags__393', 'emb_product_name_ingredients_text_brands_tags__315', 'emb_product_name_ingredients_text_brands_tags__550', 'emb_product_name_ingredients_text_brands_tags__717', 'emb_product_name_ingredients_text_brands_tags__757', 'emb_product_name_ingredients_text_brands_tags__324', 'emb_product_name_ingredients_text_brands_tags__64', 'emb_product_name_ingredients_text_brands_tags__45', 'emb_product_name_ingredients_text_brands_tags__274', 'emb_product_name_ingredients_text_brands_tags__498', 'emb_product_name_ingredients_text_brands_tags__173', 'emb_product_name_ingredients_text_brands_tags__715', 'emb_product_name_ingredients_text_brands_tags__210', 'emb_product_name_ingr

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 196221
[LightGBM] [Info] Number of data points in the train set: 81622, number of used features: 786


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 9.171473
Training until validation scores don't improve for 200 rounds
[50]	training's rmse: 8.66406	valid_1's rmse: 8.65719
[100]	training's rmse: 8.32912	valid_1's rmse: 8.32015
[150]	training's rmse: 8.01344	valid_1's rmse: 8.00235
[200]	training's rmse: 7.71541	valid_1's rmse: 7.70243
[250]	training's rmse: 7.43425	valid_1's rmse: 7.41936
[300]	training's rmse: 7.16972	valid_1's rmse: 7.15305
[350]	training's rmse: 6.92061	valid_1's rmse: 6.90218
[400]	training's rmse: 6.68639	valid_1's rmse: 6.66638
[450]	training's rmse: 6.46655	valid_1's rmse: 6.4448
[500]	training's rmse: 6.25991	valid_1's rmse: 6.23659
[550]	training's rmse: 6.06628	valid_1's rmse: 6.04158
[600]	training's rmse: 5.88465	valid_1's rmse: 5.85848
[650]	training's rmse: 5.71458	valid_1's rmse: 5.68714
[700]	training's rmse: 5.55565	valid_1's rmse: 5.52705
[750]	training's rmse: 5.40722	valid_

[7150]	training's rmse: 3.49025	valid_1's rmse: 3.50769
[7200]	training's rmse: 3.48983	valid_1's rmse: 3.50767
[7250]	training's rmse: 3.48938	valid_1's rmse: 3.50759
[7300]	training's rmse: 3.48893	valid_1's rmse: 3.50752
[7350]	training's rmse: 3.4885	valid_1's rmse: 3.5075
[7400]	training's rmse: 3.48804	valid_1's rmse: 3.5074
[7450]	training's rmse: 3.48763	valid_1's rmse: 3.50735
[7500]	training's rmse: 3.48722	valid_1's rmse: 3.50737
[7550]	training's rmse: 3.48684	valid_1's rmse: 3.50735
[7600]	training's rmse: 3.48643	valid_1's rmse: 3.50731
[7650]	training's rmse: 3.48603	valid_1's rmse: 3.50728
[7700]	training's rmse: 3.48565	valid_1's rmse: 3.50729
[7750]	training's rmse: 3.48524	valid_1's rmse: 3.50726
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7800]	training's rmse: 3.48485	valid_1's rmse: 3.50723
[LightGBM] [Warning] No further splits with positive gain, best gain:

...

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 9.170344
Training until validation scores don't improve for 200 rounds
[50]	training's rmse: 8.66146	valid_1's rmse: 8.66793
[100]	training's rmse: 8.32458	valid_1's rmse: 8.33943
[150]	training's rmse: 8.00703	valid_1's rmse: 8.03032
[200]	training's rmse: 7.70727	valid_1's rmse: 7.73857
[250]	training's rmse: 7.42448	valid_1's rmse: 7.4632
[300]	training's rmse: 7.15818	valid_1's rmse: 7.20444
[350]	training's rmse: 6.90742	valid_1's rmse: 6.96077
[400]	training's rmse: 6.67152	valid_1's rmse: 6.73206
[450]	training's rmse: 6.45009	valid_1's rmse: 6.51771
[500]	training's rmse: 6.24193	valid_1's rmse: 6.31657
[550]	training's rmse: 6.04678	valid_1's rmse: 6.12846
[600]	training's rmse: 5.86383	valid_1's rmse: 5.95184
[650]	training's rmse: 5.69243	valid_1's rmse: 5.78652
[700]	training's rmse: 5.53226	valid_1's rmse: 5.63254
[750]	training's rmse: 5.38252	valid_

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 196221
[LightGBM] [Info] Number of data points in the train set: 81623, number of used features: 786


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 9.170246
Training until validation scores don't improve for 200 rounds
[50]	training's rmse: 8.66088	valid_1's rmse: 8.669
[100]	training's rmse: 8.32513	valid_1's rmse: 8.33438
[150]	training's rmse: 8.00877	valid_1's rmse: 8.01938
[200]	training's rmse: 7.7099	valid_1's rmse: 7.72185
[250]	training's rmse: 7.42797	valid_1's rmse: 7.44136
[300]	training's rmse: 7.16261	valid_1's rmse: 7.17768
[350]	training's rmse: 6.91289	valid_1's rmse: 6.92955
[400]	training's rmse: 6.67797	valid_1's rmse: 6.69606
[450]	training's rmse: 6.45742	valid_1's rmse: 6.47712
[500]	training's rmse: 6.25017	valid_1's rmse: 6.2717
[550]	training's rmse: 6.05586	valid_1's rmse: 6.07886
[600]	training's rmse: 5.87354	valid_1's rmse: 5.89854
[650]	training's rmse: 5.70287	valid_1's rmse: 5.72969
[700]	training's rmse: 5.54328	valid_1's rmse: 5.57209
[750]	training's rmse: 5.39428	valid_1's

In [ ]:
# Saving Models


In [55]:
probs = 0
for i in models:
    probs = probs + (i.predict(df_test[feature_list]))
    print('fin_predict')
y_test_pred = probs/5.0
print(f'Real: ',math.sqrt(mean_squared_error(y_test_pred,df_test['Target'].values)))

fin_predict
fin_predict
fin_predict
fin_predict
fin_predict
Real:  4.4556653551634025
